Example

In [2]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['../src', '../example'])

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux


In [3]:
import pandas as pd
from hydroroot import radius
from hydroroot.main import hydroroot_flow
from hydroroot.init_parameter import Parameters
from hydroroot.generator.measured_root import mtg_from_aqua_data
from shared_functions import plot

In [4]:
parameter = Parameters()
parameter.read_file('parameters.yml')

In [5]:
fname = parameter.archi['input_dir'] + parameter.archi['input_file'][0]
df = pd.read_csv(fname, sep = '\t', dtype = {'order': str})
df['db'] = df['distance_from_base_(mm)'] * 1.e-3
df['lr'] = df['lateral_root_length_(mm)'] * 1.e-3

In [6]:
g = mtg_from_aqua_data(df, segment_length = parameter.archi['segment_length']) # creating the MTG from the file with a node length of 0.1 mm

In [11]:
# compute radius property on MTG
g = radius.ordered_radius(g, parameter.archi['ref_radius'], parameter.archi['order_decrease_factor'])

# compute length property and parametrisation
g = radius.compute_relative_position(g)

# radial conductivity is given in the parameters.yml as a uniform value to use hydroroot_flow we have to write 
# as 2 lists, one for the abscissa and one for the values
k = parameter.hydro['k0']
radial_conductivity_data = ([0.0, 0.2],[k,k])
g, keq, jv = hydroroot_flow(g, segment_length = parameter.archi['segment_length'], psi_e = parameter.exp['psi_e'],
                            psi_base = parameter.exp['psi_base'], axial_conductivity_data = parameter.hydro['axial_conductance_data'],
                            radial_conductivity_data = radial_conductivity_data)

In [12]:
print(keq,jv)

0.038433640014812295 0.011530092004443689


In [13]:
%gui qt
plot(g, prop_cmap='j') # j is the radial flux in ul/s